In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from glob import glob
from skimage import io

In [3]:
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
# Path to XML files
path = glob('drive/MyDrive/car_plate/images/*.xml')

# Initialize dictionary to store bounding box information
labels_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[])

# Loop through each XML file
for filename in path:
    # Parse the XML file
    info = xet.parse(filename)
    root = info.getroot()

    # Find the 'object' tag (assuming there is only one object, adjust if necessary)
    for member_object in root.findall('object'):
        labels_info = member_object.find('bndbox')
        xmin = int(labels_info.find('xmin').text)
        xmax = int(labels_info.find('xmax').text)
        ymin = int(labels_info.find('ymin').text)
        ymax = int(labels_info.find('ymax').text)

        # Normalize the file path using os.path.normpath
        normalized_filepath = os.path.normpath(filename)

        # Append the extracted data to the dictionary
        labels_dict['filepath'].append(normalized_filepath)
        labels_dict['xmin'].append(xmin)
        labels_dict['xmax'].append(xmax)
        labels_dict['ymin'].append(ymin)
        labels_dict['ymax'].append(ymax)

In [5]:
df = pd.DataFrame(labels_dict)
#df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,drive/MyDrive/car_plate/images/N102.xml,164,316,216,243
1,drive/MyDrive/car_plate/images/N100.xml,134,301,312,350
2,drive/MyDrive/car_plate/images/N101.xml,31,139,128,161
3,drive/MyDrive/car_plate/images/N1.xml,1093,1396,645,727
4,drive/MyDrive/car_plate/images/N103.xml,813,1067,665,724


In [6]:
from pathlib import Path

# Function to get the corresponding image file path from XML filename
def getFilename(filename):
    # Parse XML and extract image filename (e.g., 'N1.jpeg')
    filename_image = xet.parse(filename).getroot().find('filename').text

    # Create a Path object and join it with the directory
    filepath_image = Path('drive/MyDrive/car_plate/images') / filename_image

    # Return the path as a POSIX path (with forward slashes)
    return filepath_image.as_posix()

# Example usage:
# Apply the function to get the list of image paths with forward slashes
image_path = list(df['filepath'].apply(getFilename))

# Print the first 10 image paths
print(image_path[:10])


['drive/MyDrive/car_plate/images/N102.jpeg', 'drive/MyDrive/car_plate/images/N100.jpeg', 'drive/MyDrive/car_plate/images/N101.jpeg', 'drive/MyDrive/car_plate/images/N1.jpeg', 'drive/MyDrive/car_plate/images/N103.jpeg', 'drive/MyDrive/car_plate/images/N105.jpeg', 'drive/MyDrive/car_plate/images/N117.jpeg', 'drive/MyDrive/car_plate/images/N114.jpeg', 'drive/MyDrive/car_plate/images/N106.jpeg', 'drive/MyDrive/car_plate/images/N110.jpeg']


In [7]:
import cv2
import plotly.express as px

# Select the index of the image you want to visualize
index = 87  # Change this as needed to select a different image

# Retrieve file path and bounding box coordinates from the DataFrame
file_path = image_path[index]  # Path to the image
xmin = df['xmin'][index]        # Extract xmin from the DataFrame
ymin = df['ymin'][index]        # Extract ymin from the DataFrame
xmax = df['xmax'][index]        # Extract xmax from the DataFrame
ymax = df['ymax'][index]        # Extract ymax from the DataFrame

# Read the image using OpenCV
img = cv2.imread(file_path)

# Display the image using Plotly
fig = px.imshow(img)

# Ensure the aspect ratio is preserved and the image isn't distorted
fig.update_layout(
    autosize=False,
    width=img.shape[1],  # Width of the image
    height=img.shape[0], # Height of the image
)

# Add the bounding box from the DataFrame
fig.add_shape(
    type='rect',
    x0=xmin, x1=xmax,
    y0=ymin, y1=ymax,
    xref='x', yref='y',
    line_color='cyan'
)

# Reverse the y-axis to match OpenCV's coordinate system
fig.update_yaxes(scaleanchor="x", scaleratio=1, autorange="reversed")

# Show the image with bounding box
fig.show()


In [8]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [9]:
# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [10]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    width_shift_range=0.1,    # Randomly shift images horizontally
    height_shift_range=0.1,   # Randomly shift images vertically
    zoom_range=0.1,           # Randomly zoom into images
)

# Fit the generator to your training data
datagen.fit(x_train)



In [11]:
import tensorflow.keras.backend as K

# Custom IoU metric function
def iou_metric(y_true, y_pred):
    # Extract the bounding boxes
    x_min_true, x_max_true, y_min_true, y_max_true = y_true[:, 0], y_true[:, 1], y_true[:, 2], y_true[:, 3]
    x_min_pred, x_max_pred, y_min_pred, y_max_pred = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2], y_pred[:, 3]

    # Calculate the intersection box
    x_min_inter = K.maximum(x_min_true, x_min_pred)
    y_min_inter = K.maximum(y_min_true, y_min_pred)
    x_max_inter = K.minimum(x_max_true, x_max_pred)
    y_max_inter = K.minimum(y_max_true, y_max_pred)

    # Calculate intersection area
    inter_area = K.maximum(0.0, x_max_inter - x_min_inter) * K.maximum(0.0, y_max_inter - y_min_inter)

    # Calculate true and predicted areas
    true_area = (x_max_true - x_min_true) * (y_max_true - y_min_true)
    pred_area = (x_max_pred - x_min_pred) * (y_max_pred - y_min_pred)

    # Calculate union area
    union_area = true_area + pred_area - inter_area

    # Compute IoU
    iou = inter_area / union_area
    return iou



In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential()

# 1st Convolutional layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Convolutional layer
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Convolutional layer
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers to feed into fully connected layers
model.add(Flatten())

# Fully connected (Dense) layer
model.add(Dense(units=128, activation='relu'))

# Output layer: 4 units for bounding box coordinates (xmin, xmax, ymin, ymax)
model.add(Dense(units=4, activation='linear'))

# Compile the model with the IoU metric
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[iou_metric])

# Summary of the model
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:


# Use the generator in model training
history = model.fit(datagen.flow(x_train, y_train, batch_size=16),
                    epochs=30, validation_data=(x_test, y_test))

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - iou_metric: 0.1944 - loss: 0.0179 - val_iou_metric: 0.1732 - val_loss: 0.0224
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - iou_metric: 0.1774 - loss: 0.0153 - val_iou_metric: 0.1656 - val_loss: 0.0217
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 130ms/step - iou_metric: 0.1865 - loss: 0.0137 - val_iou_metric: 0.1376 - val_loss: 0.0223
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - iou_metric: 0.1708 - loss: 0.0148 - val_iou_metric: 0.1598 - val_loss: 0.0222
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - iou_metric: 0.1694 - loss: 0.0152 - val_iou_metric: 0.1784 - val_loss: 0.0192
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - iou_metric: 0.2076 - loss: 0.0125 - val_iou_metric: 0.1703 - val_loss: 0.0207
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - iou_metric: 0.1903 - loss: 0.0136 - val_iou_metric: 0.0524 - val_loss: 0.0362
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - iou_metric: 0.1384 - loss: 0